In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import csv
import json
import random
import time
import copy
from __future__ import print_function

%matplotlib inline

# NOTICE:
# - The fighing simulator doesn't take accuracy, status, and multi-hit moves into account.
# - Accuracy can be uncommented in attack function in Calculator
# - There are times that both pokemon uses moves that not effective against each other such as normal type move
#   against rock type pokemon making damage very low. Feel free to restart the simulation
# - When pokemon are created, it create a random pokemon with 4 random moves from its pool given by pokemonapi
# - It might take a while to create json file from pokemonapi
# - In base stat calculator, Nature value N is not taken into account as N is varied between all pokemon
#   and is very complicated to create as N is for each state for each pokemon varied from 0 to 255
# - Speed of each simulation is varied as everything is random


# ps: This is a very fun project for me. I did most of the work in 3 - 4 days. I hope you like it.
#     Feel free to tell me what I could do better.


In [2]:
typing = pd.read_csv("typing.csv")

In [3]:
print(typing.shape)
# moves.head(152)

(18, 19)


In [4]:
def create_moves_json():
    dic = {}
    print("getting moves data from https://pokeapi.co/api/v2/")
    for mv in range(1, 729):
        r = requests.get('https://pokeapi.co/api/v2/move/' + str(mv) + '/')
        name = r.json()["name"]
        dic[name] = r.json()
    with open("moves.json","w") as f:
        json.dump(dic, f)
    f.close

def create_pokemon_json():
    dic = {}
    print("getting pokemons data from https://pokeapi.co/api/v2/")
    for pkm in range(1, 808):
        r = requests.get('https://pokeapi.co/api/v2/pokemon/' + str(pkm) + '/')
        dic[str(pkm)] = r.json()
        
    with open("pokemon.json","w") as f:
        json.dump(dic, f)
    f.close
    print("pokemon.json created")
    
def load_moves_json():
    with open("moves.json") as f:
        moves = json.load(f)
    f.close
    return moves

def load_pokemon_json():
    with open("pokemon.json") as f:
        pokemon = json.load(f)
    f.close
    return pokemon

def create_typing_table():
    typing = {}
    with open('typing.csv') as csvDataFile:
        data = [row for row in csv.reader(csvDataFile)]
        header = data[0]
        for i in range(1,len(data)-1):
            tmp = {}
    #         print(header[i])
            for j in range(1,len(data[i])):
    #             print(header[j], end=' ')
                tmp[str(header[j]).lower()] = data[i][j]
    #             print(data[i][j],end=' ')
            typing[str(header[i]).lower()] = tmp
    return typing

def sortSecond(val): 
    return val[1]  

# create json file from pokemon api

# create_moves_json()
# create_pokemon_json()
pokemon = load_pokemon_json()
moves = load_moves_json()

typing = create_typing_table()

In [5]:
# print(moves.get("pound"))
# print(pokemon.get("pichu"))

# get pokemon by id
# get moves by name

In [7]:
# typing table
sspeed = 0
sspdefence = 1
sdefence = 3
sspatttack = 2
sattack = 4
shp = 5
IV = (31)/2
EV = (math.sqrt(65535)/4)/2

def get_type_effectiveness(attacking_type, defending_type):
    return typing.get(str(attacking_type)).get(str(defending_type))

def get_typing(pid):
    types_arr = pokemon.get(str(pid)).get("types")
    types = [types_arr[0].get("type").get("name")]
    if len(types_arr) == 2:
        types.append(types_arr[1].get("type").get("name"))
    return types

class Team():
    pokemons = []
    
    def __repr__(self):
        for i in self.pokemons:
            print(i.name)
        return ""
    
    def has_dup(self, pos, name):
        for i in range(6):
            if(self.pokemons[i].name == name and i != pos):
                return i
        return -1
    
    def has_dup(name):
        for i in range(6):
            if(self.pokemons[i].name == name):
                return True
        return False
    

class Calculator():
    L = 50
    
    def cal_hp(self, B, I, E):
        return math.floor(((2 * B + I + E) * self.L / 100) + self.L + 10)
    
    def cal_stat(self, B, I, E):
        # let's not take nature into account as each pokemon has different nature 
        # and each nature can increae and decrease
        # the N value which is a little too much
        N = 1
        return math.floor(math.floor((2 * B + I + E) * L / 100 + 5) * N)
    
    
    def attack(self, attacking, defending, move):
        level = 50
        accuracy = moves.get(move).get("accuracy")
        c = np.random.random()
        
#         if accuracy is not None:
#             if (c > accuracy/100.):
#                 return 0
        power = moves.get(move).get("power")
        move_type = moves.get(move).get("type").get("name")
        attacking_type = get_typing(attacking.id)
        defending_type = get_typing(defending.id)
        type_effectiveness = typing.get(move_type).get(defending_type[0])
        type_effectiveness = float(get_type_effectiveness(move_type,defending_type[0]))
        
        if(len(defending_type )== 2):
            type_effectiveness *= float(get_type_effectiveness(move_type,defending_type[1]))

        if attacking_type.count(moves.get(move).get("type").get("name")) > 0:
            stab = 1.5
        else:
            stab = 1
        modifier = round(random.uniform(0.85, 1.00),2) * stab * type_effectiveness
#         modifier = stab * type_effectiveness
        if(moves.get(move).get("damage_class").get("name") == "special"):
            A = pokemon.get(attacking.id).get("stats")[sspatttack].get("base_stat")
            D = pokemon.get(defending.id).get("stats")[sspdefence].get("base_stat")
        else:
            A = pokemon.get(attacking.id).get("stats")[sattack].get("base_stat")
            D = pokemon.get(defending.id).get("stats")[sdefence].get("base_stat")
        damage = (((((((2*level)/5)+2) * power * (A/D))/50)+2)* modifier)
        return damage
    
class Population():
    offspring = []
    teams = []
    losers = []
    fittest_team = Team()
    n = 0
    second_fitttest_team = Team()
    
    def initialize_population(self, n):
        print("initializing population...")
        self.n = n
        for i in range(n):
            self.teams.append(make_team())
    
    def repopulate(self):
        self.teams = []
        for i in range(self.n-len(self.offspring[len(self.offspring) - 1])):
            self.teams.append(make_team())
        self.teams += self.offspring[len(self.offspring) - 1]
        self.teams.append(self.fittest_team)
    
    def give_offspring(self, team_a, team_b, i, j):
        a = copy.deepcopy(team_a)
        b = copy.deepcopy(team_b)
        np.random.random()
        for pos in range(i,j + 1):
            tmp = a.pokemons[pos] # is list??
            a.pokemons[pos] = b.pokemons[pos]
            b.pokemons[pos] = tmp
            # fix where are duplicates
        if np.random.random() < 0.05:
            pos = np.random.randint(i,6)
            a.pokemons[pos] = make_random_pokemon()
        if np.random.random() < 0.05:
            pos = np.random.randint(i,6)
            b.pokemons[pos] = make_random_pokemon()
        return a, b
    
    def crossover(self):
        box = []
        for _ in range(int(math.floor(self.n * 0.2))):
            i = np.random.randint(0,6)
            j = np.random.randint(i,6)
            new_a, new_b = self.give_offspring(self.fittest_team, self.second_fitttest_team, i , j)
            box.append(new_a)
            box.append(new_b)
        self.offspring.append(box)
    
    def find_fittest(self):
        self.losers = []
        self.fittest_team = self.fittest_helper(self.teams)[0]
        b = self.losers[len(self.losers) - 1]
        self.second_fitttest_team = b[0]
        
    def fittest_helper(self, teams):
        length = len(teams)
        if length == 1:
            return teams
        elif length >= 2:
            winner_a = self.fittest_helper(teams[:length//2])
            winner_b = self.fittest_helper(teams[length//2:])
            return self.find_fitter(winner_a,winner_b)
    
    def find_fitter(self, team_a, team_b):
        a = copy.deepcopy(team_a[0])
        b = copy.deepcopy(team_b[0])
        cal = Calculator()
        i = 0
        j = 0
        a_pokemon = a.pokemons[i]
        b_pokemon = b.pokemons[j]
        score_a = 0
        score_b = 0
        while(score_a+score_b < 3):
            while(i < 6 and j < 6):
                a_speed = pokemon.get(a_pokemon.id).get("stats")[0].get("base_stat")
                b_speed = pokemon.get(b_pokemon.id).get("stats")[0].get("base_stat")
                if(a_speed > b_speed):
                    a_move = random.choice(a_pokemon.moveset)
                    b_damage = calculator.attack(a_pokemon, b_pokemon,a_move)
                    b_pokemon.take_damage(b_damage)
                    if(b_pokemon.is_dead()):
                        if j == 5: break
                        j += 1
                        b_pokemon = b.pokemons[j]
                        continue
                    b_move = random.choice(b_pokemon.moveset)
                    a_damage = calculator.attack(b_pokemon, a_pokemon,b_move)
                    a_pokemon.take_damage(a_damage)
                    if(a_pokemon.is_dead()):
                        if i == 5: break
                        i += 1
                        a_pokemon = a.pokemons[i]
                else:
                    b_move = random.choice(b_pokemon.moveset)
                    a_damage = calculator.attack(b_pokemon, a_pokemon,b_move)
                    a_pokemon.take_damage(a_damage)
                    if(a_pokemon.is_dead()):
                        if i == 5: break
                        i += 1
                        a_pokemon = a.pokemons[i]
                        continue
                    a_move = random.choice(a_pokemon.moveset)
                    b_damage = calculator.attack(a_pokemon, b_pokemon,a_move)
                    b_pokemon.take_damage(b_damage)
                    if(b_pokemon.is_dead()):
                        if j == 5: break
                        j += 1
                        b_pokemon = b.pokemons[j]
            if i > j:
                score_b += 1
            else:
                score_a += 1
                
        if score_b > score_a:
            self.losers.append(team_a)
            return team_b
        else:
            self.losers.append(team_b)
            return team_a
    
    def compare_gen(self,g1,g2):
        score_g1 = 0
        score_g2 = 0
        for i in range(200):
            t1 = random.choice(self.offspring[g1 - 1])
            t2 = random.choice(self.offspring[g2 - 1])
            winner = self.find_fitter([t1],[t2])
            if winner[0] == t1:
                score_g1 += 1
            else:
                score_g2 += 1
        return score_g1/200. , score_g2/200.
        
class Pokemon():
    id = ''
    name = ''
    moveset = []
    health = 0
    dead = False
    
    def __init__(self, id):
        self.id = id
        calculator = Calculator()
        hp = pokemon.get(str(self.id)).get("stats")[5].get("base_stat")
        self.health = calculator.cal_hp(hp,IV,EV)
#         print(self.health)

    def take_damage(self,n):
        if n >= self.health:
            self.health = 0
            self.dead = True
#             print(self.name + " is dead")
        else:
            self.health -= n
#             print(self.name + " remaining hp: " + str(self.health))
    
    def is_dead(self):
        return self.dead
    

    
def make_random_pokemon():

    pid = random.randint(1,807)
    this_pokemon = Pokemon(str(pid))
    this_pokemon.name = pokemon.get(str(pid)).get("name")
    while(len(this_pokemon.moveset) < 4):
        move = random.choice(pokemon.get(str(pid)).get("moves"))
        name = move.get("move").get("name")
        if moves.get(name).get("damage_class").get("name") == "status":
            continue
        if str(moves.get(name).get("power")) == "None":
            continue
        this_pokemon.moveset.append(name)
    return this_pokemon
    
def make_pokemon(pid):
    this_pokemon = Pokemon(str(pid))
    this_pokemon.name = pokemon.get(str(pid)).get("name")
    while(len(this_pokemon.moveset) < 4):
        move = random.choice(pokemon.get(str(pid)).get("moves"))
        name = move.get("move").get("name")
        if moves.get(name).get("damage_class").get("name") == "status":
            continue
        if str(moves.get(name).get("power")) == "None":
            continue
        this_pokemon.moveset.append(name)
    return this_pokemon


def make_team():
    random.seed(time.clock())
    team = Team()
    ar = []
    for i in range(6):
        a = make_random_pokemon()
        ar.append(a)
    team.pokemons = ar
    return team

calculator = Calculator()

def find_best_team(size_of_population,n):
    population = Population()
    population.initialize_population(size_of_population)
    population.find_fittest()
    gen = 0
    for i in range(n):
        gen += 1
        print (gen, end="\r")
        population.crossover()
        population.find_fittest()
        population.repopulate()
#     print(population.second_fitttest_team)
#     print(population.fittest_team)
    print("done all " + str(gen) + " generations")
    return population

population = find_best_team(100,100)
print(population.compare_gen(1,10))
print(population.compare_gen(1,20))
print(population.compare_gen(1,30))
print(population.compare_gen(1,40))
print(population.compare_gen(1,50))
print(population.compare_gen(1,60))
print(population.compare_gen(1,70))
print(population.compare_gen(1,80))
print(population.compare_gen(1,90))
print(population.compare_gen(1,100))
# compare_gen are more like compare offspring of 2 generations

initializing population...
done all 100 generations
(0.175, 0.825)
(0.025, 0.975)
(0.105, 0.895)
(0.06, 0.94)
(0.095, 0.905)
(0.01, 0.99)
(0.105, 0.895)
(0.065, 0.935)
(0.0, 1.0)
(0.21, 0.79)
